License

```
Copyright (c) Facebook, Inc. and its affiliates.

This source code is licensed under the MIT license found in the
LICENSE file in the root directory of this source tree.
```

# CompilerGym Getting Started

CompilerGym is a toolkit for applying reinforcement learning to compiler optimization tasks. This document provides a short walkthrough of the key concepts, using the codesize reduction task of a production-grade compiler as an example. It will take about 20 minutes to work through. Lets get started!

## Key Concepts

CompilerGym exposes compiler optimization problems as environments for reinforcement learning. It uses the [OpenAI Gym](https://gym.openai.com/) interface to expose the “agent-environment loop” of reinforcement learning:

![overview](https://facebookresearch.github.io/CompilerGym/_images/overview.png)

The ingredients for reinforcement learning that CompilerGym provides are:

* **Environment**: a compiler optimization task. For example, *optimizing a C++ graph-traversal program for codesize using LLVM*. The environment encapsulates an instance of a compiler and a particular program that is being compiled. As an agent interacts with the environment, the state of the program, and the compiler, can change.
* **Action Space**: the actions that may be taken at the current environment state. For example, this could be a set of optimization transformations that the compiler can apply to the program.
* **Observation**: a view of the current environment state. For example, this could be the Intermediate Representation (IR) of the program that is being compiled. The types of observations that are available depend on the compiler.
* **Reward**: a metric indicating the quality of the previous action. For example, for a codesize optimization task this could be the change to the number of instructions of the previous action.

A single instance of this “agent-environment loop” represents the compilation of a particular program. The goal is to develop an agent that maximises the cumulative reward from these environments so as to produce the best programs.

## Installation

Install the latest CompilerGym release using:

In [ ]:
!pip install compiler_gym

See [INSTALL.md](https://github.com/facebookresearch/CompilerGym/blob/development/INSTALL.md) for alternative installation methods.

## Using CompilerGym

To start with we import the gym module and the CompilerGym environments:

In [ ]:
import gym
import compiler_gym

compiler_gym.__version__

Importing `compiler_gym` automatically registers the compiler environments.

We can see what environments are available using:

In [ ]:
compiler_gym.COMPILER_GYM_ENVS

## Selecting an environment

CompilerGym environments are named using one of the following formats:

* `<compiler>-<observation>-<reward>-<version>`
* `<compiler>-<reward>-<version>`
* `<compiler>-<version>`

Where `<compiler>` identifiers the compiler optimization task, `<observation>` is the default type of observations that are provided, and `<reward>` is the reward signal.

**Note** A key concept is that CompilerGym environments enables **lazy evaluation** of observations and reward signals. This increases computational efficiency sampling for scenarios in which you do not need to compute a reward or observation for every step. If an environment omits a `<observation>` or `<reward>` tag, this means that no observation or reward is provided by default. See [compiler_gym.views](https://facebookresearch.github.io/CompilerGym/compiler_gym/views.html) for further details.

For this tutorial, we will use the following environment:
* **Compiler**: [LLVM](https://facebookresearch.github.io/CompilerGym/llvm/index.html).
* **Observation Type**: [Autophase](https://facebookresearch.github.io/CompilerGym/llvm/index.html#autophase).
* **Reward Signal**: [IR Instruction count relative to -Oz](https://facebookresearch.github.io/CompilerGym/llvm/index.html#codesize).

Create an instance of this environment using:

In [ ]:
env = gym.make("llvm-autophase-ic-v0")

**Note** The first time you run `gym.make()` you may see a logging message "Downloading \<url\> ..." followed by a delay of 1-2 minutes. This is CompilerGym downloading large environment-specific dependencies that are not shipped by default to keep the size of the package down. This is a one-off download that occurs only the first time the environment is used. Other operations that require one-off downloads include installing datasets (described below).

## The compiler environment

If you have experience using [OpenAI Gym](https://gym.openai.com/), the CompilerGym environments will be familiar. If not, you can call `help()` on any function, object or method to query the documentation:

In [ ]:
help(env.step)

The action space is described by `env.action_space`. The [LLVM Action Space](https://facebookresearch.github.io/CompilerGym/llvm/index.html#action-space) is discrete:

In [ ]:
env.action_space.dtype

In [ ]:
env.action_space.n

The observation space is described by `env.observation_space`. The [Autophase](https://facebookresearch.github.io/CompilerGym/llvm/index.html#autophase) observation space is a 56-dimension vector of integers:

In [ ]:
env.observation_space.shape

In [ ]:
env.observation_space.dtype

The upper and lower bounds of the reward signal are described by `env.reward_range`:

In [ ]:
env.reward_range

As with other Gym environments, `reset()` must be called before a CompilerGym environment may be used:

In [ ]:
env.reset()

The numpy array that is returned by `reset()` is the initial observation. This value, along with the entire dynamics of the environment, depend on the particular program that is being compiled. In CompilerGym these programs are called **benchmarks**. You can see which benchmark is currently being used by an environment using `env.benchmark`:

In [ ]:
env.benchmark

If we want to compile a different program, we can pass the name of a benchmark to `env.reset()`:

In [ ]:
env.reset(benchmark="benchmark://npb-v0/50")

We provide over [a million benchmarks for the LLVM environments](https://facebookresearch.github.io/CompilerGym/llvm/index.html#datasets) that can be used for training agents and evaluating the
generalization of strategies across unseen programs. Benchmarks are grouped into *datasets* , which are managed using `env.datasets`. You may also provide your own programs to use as benchmarks, see `env.make_benchmark()` for details.

## Interacting with the environment

Once an environment has been initialized, you interact with it in the same way that you would with any other [OpenAI Gym](https://gym.openai.com/) environment. `env.render()` prints the Intermediate Representation (IR) of the program in the current state:

In [ ]:
env.render()

`env.step()` runs an action:

In [ ]:
observation, reward, done, info = env.step(0)

This returns four values: a new observation, a reward, a boolean value indicating whether the episode has ended, and a dictionary of additional information:

In [ ]:
observation

In [ ]:
reward

In [ ]:
done

In [ ]:
info

For this environment, reward represents the reduction in code size of the 
previous action, scaled to the total codesize reduction achieved with LLVM's `-Oz` optimizations enabled. A cumulative reward greater than one means that the sequence of optimizations performed yields better results than LLVM's default optimizations. Let's run 100 random actions and see how close we can  get:

In [ ]:
env.reset(benchmark="benchmark://npb-v0/50")
episode_reward = 0
for i in range(1, 101):
    observation, reward, done, info = env.step(env.action_space.sample())
    if done:
        break
    episode_reward += reward
    print(f"Step {i}, quality={episode_reward:.2%}")

Not bad, but clearly there is room for improvement! Because at each step we are taking random actions, your results will differ with every run. Try running it again. Was the result better or worse? Of course, there may be better ways of selecting actions than choosing randomly, but for the purpose of this tutorial we will leave that as an exercise for the reader :)

Before we finish, lets use `env.commandline()` to produce an LLVM `opt` command line invocation that is equivalent to the sequence of actions we just run:

In [ ]:
env.commandline()

We can also save the program in its current state for future reference:

In [ ]:
env.write_bitcode("/tmp/program.bc")

!ls /tmp/program.bc

Once we are finished, we must close the environment to end the compiler session:

In [ ]:
env.close()

## Next Steps

Now that you have got to grips with the compiler environment, take a browse through the [examples directory](https://github.com/facebookresearch/CompilerGym/tree/stable/examples) for pytorch integration, agent implementations, etc. Then check out [the leaderboards](https://github.com/facebookresearch/CompilerGym#leaderboards) to see what the best performing algorithms are, and [the documentation](https://facebookresearch.github.io/CompilerGym/) for details of the APIs and environments. We love feedback, bug reports, and feature requests - please [file an issue](https://github.com/facebookresearch/CompilerGym/issues/new/choose)!